# Colour Correction using Splines

### Define constants

In [23]:
# Enable automatic reloading of modules before executing code to ensure latest changes are used.
%load_ext autoreload
%autoreload 2

# Importing necessary libraries and functions.
# 'colour' is used for color science computations.
# 'data' contains functions to load various datasets and color matching functions.
import colour
from data import (load_dataset_sfu, load_dataset_csv, load_cmfs, 
                  load_camera, load_insitu, msds_to_rgb, msds_to_xyz, load_dataset_skin)
import numpy as np

# Setting a seed for random number generation to ensure reproducibility.
RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)

# Defining file paths for the datasets. These files should contain the data needed for analysis.
SFU_FILE_PATH = 'data/reflect_db.reflect'  # SFU material reflectance database
CAVE_FOSTER2004_PATH = 'data/cave_foster2002.csv'  # CAVE dataset for color constancy
INSITU_PATH = "data/insitu_dataset.csv"  # In-situ measurements dataset
CAVE_PATH = 'data/cave.csv'  # Another CAVE dataset file
FOSTER_50_PATH = 'data/foster50.csv'  # Foster dataset with 50 images for color analysis
CAMERA = 'nikon'  # Specifying the camera model used in the datasets or for simulation


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
cave_foster2004_dataset = load_dataset_csv(CAVE_FOSTER2004_PATH)
foster_50_dataset = load_dataset_csv(FOSTER_50_PATH)
insitu = load_insitu(INSITU_PATH)


In [25]:
TRAIN = cave_foster2004_dataset
TEST = foster_50_dataset
VALIDATION = insitu

### Computing Observer Responses
We can easily change the order of test and train sets here

In [26]:
from plotting import plot_chromaticity_diagram
from colour.characterisation import training_data_sds_to_XYZ

cmfs, illuminant = load_cmfs()
response_trainset_xyz = msds_to_xyz(TRAIN, cmfs, illuminant)
response_testset_xyz = msds_to_xyz(TEST, cmfs, illuminant)
response_validation_xyz = msds_to_xyz(VALIDATION, cmfs, illuminant)


[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]
[ 0.94940092  1.          1.08709122]


### Computing Camera Responses

In [27]:
from colour.characterisation import normalise_illuminant, training_data_sds_to_RGB
import numpy as np
MSDS_TRAIN = load_camera(CAMERA)

response_trainset_camera = msds_to_rgb(TRAIN,MSDS_TRAIN, illuminant)
response_testset_camera = msds_to_rgb(TEST,MSDS_TRAIN, illuminant)
response_validationset_camera = msds_to_rgb(VALIDATION,MSDS_TRAIN, illuminant)


[  613.25528366  1055.57702291   900.69359634]
[  613.25528366  1055.57702291   900.69359634]
[  613.25528366  1055.57702291   900.69359634]


### Fit Generalized Additive Model with P-splines

### Nikon

In [28]:
from models import GAMOptimizer
from evaluate import pred
def test_gam_optimizer(splines_lams_dict, order_value=3):
    """
    Test GAMOptimizer models with different numbers of splines and corresponding lambda values.

    Parameters:
    - splines_lams_dict: Dictionary where keys are spline numbers and values are the corresponding lambda values for regularization.
    - order_value: The order of the spline. Default is 3.
    """
    for n_splines, lams_value in splines_lams_dict.items():
        print(f"Testing GAMOptimizer with {n_splines} splines and lambda {lams_value}")
        gam = GAMOptimizer(lams=lams_value, order=order_value, n_splines=n_splines)
        gam.fit(response_trainset_camera, response_trainset_xyz)
        pred(gam, response_testset_camera, response_testset_xyz, f"DeltaE Foster+CAVE with {n_splines} splines and lambda {lams_value}")

# Example usage: Create a dictionary mapping spline numbers to their corresponding lambda values and pass it to the function.
splines_lams_dict = {
    5: 1e-6,
    10: 0.0001,
    20: 0.01
}
if CAMERA == 'nikon':
    test_gam_optimizer(splines_lams_dict)

Testing GAMOptimizer with 5 splines and lambda 1e-06
225
---- RESULTS DeltaE Foster+CAVE with 5 splines and lambda 1e-06 ----
DeltaE mean: 0.94
DeltaE max: 4.81
DeltaE median: 0.81
DeltaE 95 percentile: 2.08
DeltaE 99 percentile: 2.48
Testing GAMOptimizer with 10 splines and lambda 0.0001
900
---- RESULTS DeltaE Foster+CAVE with 10 splines and lambda 0.0001 ----
DeltaE mean: 0.86
DeltaE max: 5.16
DeltaE median: 0.73
DeltaE 95 percentile: 1.94
DeltaE 99 percentile: 2.45
Testing GAMOptimizer with 20 splines and lambda 0.01
3600
---- RESULTS DeltaE Foster+CAVE with 20 splines and lambda 0.01 ----
DeltaE mean: 0.83
DeltaE max: 5.77
DeltaE median: 0.71
DeltaE 95 percentile: 1.92
DeltaE 99 percentile: 2.42


### Sigma

In [29]:
def test_gam_optimizer(splines_lams_dict, order_value=3):
    """
    Test GAMOptimizer models with different numbers of splines and corresponding lambda values.

    Parameters:
    - splines_lams_dict: Dictionary where keys are spline numbers and values are the corresponding lambda values for regularization.
    - order_value: The order of the spline. Default is 3.
    """
    for n_splines, lams_value in splines_lams_dict.items():
        print(f"Testing GAMOptimizer with {n_splines} splines and lambda {lams_value}")
        gam = GAMOptimizer(lams=lams_value, order=order_value, n_splines=n_splines)
        gam.fit(response_trainset_camera, response_trainset_xyz)
        pred(gam, response_testset_camera, response_testset_xyz, f"{n_splines} splines and lambda {lams_value}")

# Example usage: Create a dictionary mapping spline numbers to their corresponding lambda values and pass it to the function.
splines_lams_dict = {
    5: 1e-9,
    10: 0.0001,
    20: 0.0001
}
if CAMERA == 'sigma':
    test_gam_optimizer(splines_lams_dict)

In [30]:
from models import RGBtoXYZNetwork

mcdonalds_nn = RGBtoXYZNetwork()
mcdonalds_nn.fit(response_trainset_camera, response_trainset_xyz)
pred(mcdonalds_nn, response_testset_camera, response_testset_xyz, "Foster 50")

Epoch 1, Loss: 0.0055784424766898155
Epoch 2, Loss: 0.005691935773938894
Epoch 3, Loss: 0.0007155074272304773
Epoch 4, Loss: 0.0007036969182081521
Epoch 5, Loss: 0.0016094653401523829
Epoch 6, Loss: 0.0002593286626506597
Epoch 7, Loss: 0.00022408370568882674
Epoch 8, Loss: 0.00010509035200811923
Epoch 9, Loss: 0.0014765344094485044
Epoch 10, Loss: 7.533961615990847e-05
Epoch 11, Loss: 7.688671030336991e-05
Epoch 12, Loss: 0.00026393186999484897
Epoch 13, Loss: 0.00014708584058098495
Epoch 14, Loss: 4.673297735280357e-05
Epoch 15, Loss: 1.6646559743094258e-05
Epoch 16, Loss: 0.00020208604109939188
Epoch 17, Loss: 1.6204572602873668e-05
Epoch 18, Loss: 2.9028558856225573e-05
Epoch 19, Loss: 2.6047468054457568e-05
Epoch 20, Loss: 1.9646528016892262e-05
Epoch 21, Loss: 1.4616281987400725e-05
Epoch 22, Loss: 1.4016002751304768e-05
Epoch 23, Loss: 2.885338744818e-06
Epoch 24, Loss: 5.618491286440985e-06
Epoch 25, Loss: 1.266285926249111e-05
Epoch 26, Loss: 1.014912504615495e-05
Epoch 27, Los

### Fit Linear Model

In [31]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression(fit_intercept=False)


linear.fit(response_trainset_camera, response_trainset_xyz)
pred(linear, response_testset_camera, response_testset_xyz, "Foster 50")


---- RESULTS Foster 50 ----
DeltaE mean: 1.00
DeltaE max: 5.56
DeltaE median: 0.86
DeltaE 95 percentile: 2.18
DeltaE 99 percentile: 2.93


In [32]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=1))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.8916868419873166
        x: [ 6.535e-01  2.491e-01  4.907e-02  2.453e-01  1.037e+00
            -2.827e-01  4.326e-02 -2.874e-01  1.346e+00]
      nit: 27
      jac: [-6.981e-06 -5.931e-06 -5.685e-06  8.203e-06  6.892e-06
             6.199e-06 -3.427e-07 -5.737e-07  3.800e-07]
 hess_inv: [[ 2.957e-02 -5.076e-02 ..., -3.721e-02  1.987e-02]
            [-5.076e-02  1.424e-01 ...,  1.112e-01 -7.681e-02]
            ..., 
            [-3.721e-02  1.112e-01 ...,  1.744e-01 -1.334e-01]
            [ 1.987e-02 -7.681e-02 ..., -1.334e-01  1.311e-01]]
     nfev: 410
     njev: 41
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 0.94
DeltaE max: 5.51
DeltaE median: 0.81
DeltaE 95 percentile: 2.04
DeltaE 99 percentile: 2.66


### Fit 3rd order Root-Polynomial Model

In [33]:
from models import PolynomialTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])

RP_linear_3.fit(response_trainset_camera, response_trainset_xyz)

pred(RP_linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.06
DeltaE max: 5.69
DeltaE median: 0.85
DeltaE 95 percentile: 2.43
DeltaE 99 percentile: 3.00


### Fit a 2nd order Root-Polynomial Model

In [34]:
from models import GAMOptimizer, PolynomialTransformer, DeltaEOptimizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_2 = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])


RP_linear_2.fit(response_trainset_camera, response_trainset_xyz)
pred(RP_linear_2, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.05
DeltaE max: 5.54
DeltaE median: 0.84
DeltaE 95 percentile: 2.44
DeltaE 99 percentile: 3.06


In [35]:
P_Linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_Linear_3.fit(response_trainset_camera, response_trainset_xyz)
pred(P_Linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.07
DeltaE max: 5.40
DeltaE median: 0.90
DeltaE 95 percentile: 2.37
DeltaE 99 percentile: 2.98


In [36]:
P_linear = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_linear.fit(response_trainset_camera, response_trainset_xyz)
pred(P_linear, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 1.13
DeltaE max: 5.58
DeltaE median: 0.98
DeltaE 95 percentile: 2.43
DeltaE 99 percentile: 3.05


In [37]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000P = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=3))
])


DE2000P.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000P, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.6267925301544306
        x: [ 6.477e-01  2.760e-01 ...,  5.061e-01 -2.041e+00]
      nit: 292
      jac: [-2.183e-06 -3.971e-06 ..., -7.823e-07  6.862e-06]
 hess_inv: [[ 1.050e-01 -1.371e-01 ...,  8.229e-01 -5.343e-01]
            [-1.371e-01  4.222e-01 ..., -2.687e+00  3.698e+00]
            ..., 
            [ 8.229e-01 -2.687e+00 ...,  6.292e+02 -7.718e+02]
            [-5.343e-01  3.698e+00 ..., -7.718e+02  2.023e+03]]
     nfev: 17632
     njev: 304
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 0.91
DeltaE max: 7.07
DeltaE median: 0.77
DeltaE 95 percentile: 2.03
DeltaE 99 percentile: 2.49


In [38]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=True, degree=3))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.6515607986652309
        x: [ 2.071e-01 -8.932e+00 ...,  1.128e+01 -5.832e+00]
      nit: 370
      jac: [ 5.193e-06  8.725e-06 ...,  8.568e-07 -9.686e-08]
 hess_inv: [[ 3.512e+02  3.149e+02 ..., -5.469e+02 -9.327e+01]
            [ 3.149e+02  2.566e+03 ..., -1.380e+03  1.681e+03]
            ..., 
            [-5.469e+02 -1.380e+03 ...,  9.717e+03 -1.981e+02]
            [-9.327e+01  1.681e+03 ..., -1.981e+02  5.972e+03]]
     nfev: 15920
     njev: 398
---- RESULTS DeltaE Foster+CAVE ----
DeltaE mean: 0.92
DeltaE max: 8.04
DeltaE median: 0.76
DeltaE 95 percentile: 2.22
DeltaE 99 percentile: 3.04
